In [1]:
# Import libs
import sys
import matplotlib.pyplot as plt
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.relax_linear import LinearRelaxer
import itertools

try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

In [2]:
SIN_PATA = "SIN_PATA"
SIN_DOS_PATAS = "SIN_DOS_PATAS"
SIN_ASIENTO = "SIN_ASIENTO"
SIN_RESPALDO = "SIN_RESPALDO"
SIN_RESPALDO_PATA = "SIN_RESPALDO_PATA"
PATAS = "PATAS"
ASIENTOS = "ASIENTOS"
RESPALDOS = "RESPALDOS"
SILLA = "SILLA"
TABURETE_3_PATAS = "TABURETE_3_PATAS"
TABURETE_4_PATAS = "TABURETE_4_PATAS"

In [3]:
# CANTIDAD DE SILLAS PARA REGALAR
SILLAS_PARA_FLACA = 1

# COMPRA
# TABURETE, ASIENTOS, RESPALDO
P, A, R = 20, 40, 45

# VENTA
# SILLA, TABURETE 4 PATAS, TABURETE 3 PATAS
P1, P2, P3 = 100, 70, 50

# DEMANDA
# SILLA, TABURETE 4 PATAS, TABURETE 3 PATAS
D1, D2, D3 = 50, 80, 70

# RECURSOS
sillas_rotas = {
    SIN_PATA: 20,     
    SIN_DOS_PATAS: 10,     
    SIN_ASIENTO: 10,     
    SIN_RESPALDO: 30,   
    SIN_RESPALDO_PATA: 30     
}

# STOCK
patas_stock = (
    4 * sillas_rotas[SIN_ASIENTO] + 
    4 * sillas_rotas[SIN_RESPALDO] + 
    3 * sillas_rotas[SIN_PATA] + 
    2 * sillas_rotas[SIN_DOS_PATAS] + 
    3 * sillas_rotas[SIN_RESPALDO_PATA]
)

asientos_stock = (
    sillas_rotas[SIN_DOS_PATAS] + 
    sillas_rotas[SIN_RESPALDO] + 
    sillas_rotas[SIN_PATA] + 
    sillas_rotas[SIN_RESPALDO_PATA]
)

respaldos_stock = (
    sillas_rotas[SIN_ASIENTO] + 
    sillas_rotas[SIN_PATA] + 
    sillas_rotas[SIN_DOS_PATAS]
)

In [4]:
mdl = Model(name="Ejericio 2.32: Muebles")

sillas_fabricadas = mdl.integer_var(name="sillas fabricadas")
taburete_4_fabricados = mdl.integer_var(name="taburete2 (4) fabricados")
taburete_3_fabricados = mdl.integer_var(name="taburetes (3) fabricados")

patas_compradas = mdl.integer_var(name="sillas compradas")
asientos_comprados = mdl.integer_var(name="asientos comprados")
respaldos_comprados = mdl.integer_var(name="respaldos comprados")

# RESTRICCIONES

# SATISFACER DEMANDA
mdl.add_constraint(sillas_fabricadas >= D1)
mdl.add_constraint(taburete_4_fabricados >= D2)
mdl.add_constraint(taburete_3_fabricados >= D3)

# RELACION STOCK/COMPRAS CON FABRICADOS 
mdl.add_constraint(patas_stock + patas_compradas == 4*sillas_fabricadas + 3*taburete_3_fabricados + 4*taburete_4_fabricados + 4*SILLAS_PARA_FLACA)      
mdl.add_constraint(asientos_stock + asientos_comprados == sillas_fabricadas + taburete_3_fabricados + taburete_4_fabricados + SILLAS_PARA_FLACA)
mdl.add_constraint(respaldos_stock + respaldos_comprados == sillas_fabricadas + SILLAS_PARA_FLACA)

# FUNCION OBJETIVO
ganancia = P1*sillas_fabricadas + P2*taburete_4_fabricados + P3*taburete_3_fabricados
costo = P*patas_compradas + A*asientos_comprados + R*respaldos_comprados
beneficio = ganancia - costo
mdl.maximize(beneficio)

solution = mdl.solve()

In [5]:
# Imprimir la solución
if solution:
    print(f'BENEFICIO:')
    print(f' - {mdl.objective_value}$')

    print('\nFABRICADO:')
    print(f' - Sillas: {sillas_fabricadas.solution_value}')
    print(f' - Taburetes de cuatro patas: {taburete_4_fabricados.solution_value}')
    print(f' - Taburetes de tres patas: {taburete_3_fabricados.solution_value}')
    print(f'(Total muebles realizados: {sillas_fabricadas.solution_value + taburete_3_fabricados.solution_value + taburete_4_fabricados.solution_value}, 
          Total: {P1*sillas_fabricadas.solution_value + P2*taburete_4_fabricados.solution_value + P3*taburete_3_fabricados.solution_value}$)')
    
    print("\nCOMPRADO:")
    print(f' - Patas: {patas_compradas.solution_value}')
    print(f' - Asientos: {asientos_comprados.solution_value}')
    print(f' - Respaldos: {respaldos_comprados.solution_value}')
    print(f'(Total piezas compradas: {patas_compradas.solution_value + asientos_comprados.solution_value + respaldos_comprados.solution_value}, 
          Total: -{P*patas_compradas.solution_value + A*asientos_comprados.solution_value + R*respaldos_comprados.solution_value}$)')

else:
    print("No se encontro solución.")

BENEFICIO:
 - 1085.0$

FABRICADO:
 - Sillas: 50.0
 - Taburetes de cuatro patas: 80.0
 - Taburetes de tres patas: 70.0
(Total muebles realizados: 200.0, Total: 14100.0$)

COMPRADO:
 - Patas: 404.0
 - Asientos: 111.0
 - Respaldos: 11.0
(Total piezas compradas: 526.0, Total: -13015.0$)
